In [ ]:
from preprocess import preprocess_data_flat
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import f1_score, make_scorer, accuracy_score, balanced_accuracy_score, roc_auc_score
from tabulate import tabulate 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

In [6]:
df_final = [preprocess_data_flat(x, 30) for x in ["1", "2"]]
x_final, y_final = [pd.concat([x[i] for x in df_final], ignore_index=True) for i in range(2)]
X_train_flat, X_test_flat, y_train_flat, y_test_flat = train_test_split(
    x_final.to_numpy(), y_final.to_numpy(), test_size=0.2, random_state=123
)

In [ ]:
model = ExtraTreesClassifier(
    random_state=123,
)

param_dist = {
    "max_depth": [2, 3, 4],
    "n_estimators": [100, 200, 300, 400],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", None],
    "bootstrap": [True, False],
    "class_weight": ["balanced", "balanced_subsample", None],
    "criterion": ["gini", "entropy", "log_loss"],
}

n_splits = 5
cv = TimeSeriesSplit(n_splits=n_splits)

random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    scoring="balanced_accuracy",
    cv=cv,
    random_state=123,
    n_jobs=10,
    refit="balanced_accuracy",
    verbose=0,
    n_iter=100,
)

random_search.fit(X_train_flat, y_train_flat)
best_model = random_search.best_estimator_

In [8]:
scoring = {
    "accuracy": make_scorer(accuracy_score),
    "balanced_accuracy": make_scorer(balanced_accuracy_score),
    "roc_auc": make_scorer(roc_auc_score),
    "f1": make_scorer(f1_score),
}


cv_results = cross_validate(
    best_model,
    X_train_flat,
    y_train_flat,
    cv=cv,
    scoring=scoring,
    return_train_score=True,
    n_jobs=-1,
)


fold_results = []
for i in range(n_splits):
    fold_results.append(
        {
            "Fold": i + 1,
            "Train Acc": f"{cv_results['train_accuracy'][i]:.4f}",
            "Test Acc": f"{cv_results['test_accuracy'][i]:.4f}",
            "Train BA": f"{cv_results['train_balanced_accuracy'][i]:.4f}",
            "Test BA": f"{cv_results['test_balanced_accuracy'][i]:.4f}",
            "Train AUC": f"{cv_results['train_roc_auc'][i]:.4f}",
            "Test AUC": f"{cv_results['test_roc_auc'][i]:.4f}",
            "Train F1": f"{cv_results['train_f1'][i]:.4f}",
            "Test F1": f"{cv_results['test_f1'][i]:.4f}",
            "Time (s)": f"{cv_results['fit_time'][i] + cv_results['score_time'][i]:.3f}",
        }
    )

summary_results = {
    "F1 Score": f"{cv_results['test_f1'].mean():.4f} ± {cv_results['test_f1'].std():.4f}",
    "Accuracy": f"{cv_results['test_accuracy'].mean():.4f} ± {cv_results['test_accuracy'].std():.4f}",
    "Balanced Accuracy": f"{cv_results['test_balanced_accuracy'].mean():.4f} ± {cv_results['test_balanced_accuracy'].std():.4f}",
    "ROC AUC": f"{cv_results['test_roc_auc'].mean():.4f} ± {cv_results['test_roc_auc'].std():.4f}",
    "Total Time (s)": f"{(cv_results['fit_time'] + cv_results['score_time']).sum():.3f}",
}

print("\n" + "=" * 80)
print("CROSS-VALIDATION RESULTS")
print("=" * 80)

print("\n RESULTS BY FOLD:")
print(tabulate(fold_results, headers="keys", tablefmt="pretty"))

print("\n STATISTICS (Test Set within CV):")
for metric, value in summary_results.items():
    print(f"   • {metric:<20}: {value}")


print("\n" + "=" * 80)
print("FINAL EVALUATION ON HELD-OUT TEST SET")
print("=" * 80)
y_pred_test = best_model.predict(X_test_flat)
y_prob_test = best_model.predict_proba(X_test_flat)[:, 1]

test_f1 = f1_score(y_test_flat, y_pred_test)
test_accuracy = accuracy_score(y_test_flat, y_pred_test)
test_balanced_accuracy = balanced_accuracy_score(y_test_flat, y_pred_test)
test_roc_auc = roc_auc_score(y_test_flat, y_prob_test)

print(f"   • {'F1 Score':<20}: {test_f1:.4f}")
print(f"   • {'Accuracy':<20}: {test_accuracy:.4f}")
print(f"   • {'Balanced Accuracy':<20}: {test_balanced_accuracy:.4f}")
print(f"   • {'ROC AUC':<20}: {test_roc_auc:.4f}")
print("=" * 80)


CROSS-VALIDATION RESULTS

 RESULTS BY FOLD:
+------+-----------+----------+----------+---------+-----------+----------+----------+---------+----------+
| Fold | Train Acc | Test Acc | Train BA | Test BA | Train AUC | Test AUC | Train F1 | Test F1 | Time (s) |
+------+-----------+----------+----------+---------+-----------+----------+----------+---------+----------+
|  1   |  1.0000   |  0.8361  |  1.0000  | 0.7707  |  1.0000   |  0.7707  |  1.0000  | 0.8889  |  0.500   |
|  2   |  0.9762   |  0.8525  |  0.9583  | 0.8064  |  0.9583   |  0.8064  |  0.9836  | 0.8966  |  0.569   |
|  3   |  0.9626   |  0.8033  |  0.9364  | 0.7661  |  0.9364   |  0.7661  |  0.9742  | 0.8537  |  0.723   |
|  4   |  0.9758   |  0.8689  |  0.9658  | 0.8461  |  0.9658   |  0.8461  |  0.9824  | 0.8974  |  0.840   |
|  5   |  0.9515   |  0.9508  |  0.9309  | 0.8929  |  0.9309   |  0.8929  |  0.9644  | 0.9691  |  1.022   |
+------+-----------+----------+----------+---------+-----------+----------+----------+-----